In [1]:
import sys
import json
import time
from collections import defaultdict
import pandas as pd
import os
import numpy as np

sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/dgl_graphsage')
from src.api.spotifyAPI import SpotifyAPI
from utils import load_data
from train_updated import train

import dgl
import torch
import torch.nn.functional as F
from dgl import save_graphs
from dgl import load_graphs

from sklearn.neighbors import NearestNeighbors

Using backend: pytorch


# Load Data

In [2]:
# %%time
# # Loading in the data. ~ 00:01:30
# with open('config/data-params.json') as fh:
#     data_cfg = json.load(fh)
# feat_data, adj_list, dgl_G, uri_map = load_data(**data_cfg)

### From Scratch

In [3]:
sys.path.insert(0, './src/features')
from build_features import load_data as graph_from_scratch
import re
from torch.nn.functional import normalize

In [4]:
# uses original files to make the graph from scratch

# def scratch(feat_dir, normal=True):
#     print('Loading feature data...')
#     data = np.genfromtxt(feat_dir, delimiter=',', skip_header=True, dtype=str)
#     features = np.array(np.delete(data[:,2:], -3, 1), dtype=float)
#     if normal:
#         features = normalize(torch.Tensor(features), dim=0)
#     uris = data[:, 1]
#     uris = [re.sub('spotify:track:', '', uri) for uri in uris]
#     uri_map = {n: i for i,n in enumerate(uris)}
    
#     G = graph_from_scratch('./data/a13group1/data/', 'Spotify Playlist', 0, 0, 0)
#     print('graph created')
#     src, dest = [], [] 
#     adj_list = defaultdict(set)    
#     for e in G.edges:
#         u,v = uri_map[e[0]], uri_map[e[1]]
#         adj_list[u].add(v)
#         adj_list[v].add(u)
#         src.append(u)
#         dest.append(v)
        
#     src = torch.tensor(src)
#     dest = torch.tensor(dest)
        
    
#     print('adj list created')
    
#     dgl_G = dgl.graph((src, dest), num_nodes=len(G.nodes))
#     return features, adj_list, dgl_G, uri_map

In [5]:
# '''
# Creates graph from scratch using from_networkx
# results in double edges
# '''
# def scratch(feat_dir, normal=True):
#     print('Loading feature data...')
#     data = np.genfromtxt(feat_dir, delimiter=',', skip_header=True, dtype=str)
#     features = np.array(np.delete(data[:,2:], -3, 1), dtype=float)
#     if normal:
#         features = normalize(torch.Tensor(features), dim=0)
#     uris = data[:, 1]
#     uris = [re.sub('spotify:track:', '', uri) for uri in uris]
#     uri_map = {n: i for i,n in enumerate(uris)}
    
#     G = graph_from_scratch('./data/a13group1/data/', 'Spotify Playlist', 0, 0, 0)
#     print('graph created')
#     src, dest = [], [] 
#     adj_list = defaultdict(set)    
#     for e in G.edges:
#         u,v = uri_map[e[0]], uri_map[e[1]]
#         adj_list[u].add(v)
#         adj_list[v].add(u)
#         src.append(u)
#         dest.append(v)
#     print('adj list created')
    
#     #dgl_G = dgl.graph((src, dest), num_nodes=len(G.nodes))
#     dgl_G = dgl.from_networkx(G)
#     return features, adj_list, dgl_G, uri_map

### Currently the one I am using

In [13]:
'''
Loads the double edged graph
feat_dir = feature directory
'''
def load_double_edge(feat_dir, normal=True):
    print('Loading feature data...')
    data = np.genfromtxt(feat_dir, delimiter=',', skip_header=True, dtype=str)
    features = np.array(np.delete(data[:,2:], -3, 1), dtype=float)
    if normal:
        features = normalize(torch.Tensor(features), dim=0)
    uris = data[:, 1]
    uris = [re.sub('spotify:track:', '', uri) for uri in uris]
    uri_map = {n: i for i,n in enumerate(uris)}
    listed = list(uri_map)
    
    G  = load_graphs("./data/a13group1/double_edges_170k.bin")[0][0]
    print('Loaded DGL Graph')
    sources = G.edges()[0] 
    destinations = G.edges()[1]
    
    src, dest = [], [] 
    adj_list = defaultdict(set)
    for e in range(len((G.edges()[0]))):
        u,v = sources[e].item(), destinations[e].item()
        adj_list[u].add(v)
        adj_list[v].add(u)
        src.append(u)
        dest.append(v)
        
    
    print('adj list created')
    return features, adj_list, G, uri_map

In [14]:
#%%time
#feat_data, adj_list, dgl_G, uri_map = scratch("./data/a13group1/features/merged_features.csv")

CPU times: user 0 ns, sys: 28 µs, total: 28 µs
Wall time: 54.4 µs


In [15]:
# Saving Graph if built from scratch
# save_graphs("./data/a13group1/double_edges_170k.bin", dgl_G)

In [16]:
# Loading Graph ~ 3min
%%time
feat_data, adj_list, dgl_G, uri_map = load_double_edge("./data/a13group1/features/merged_features.csv")

Loading feature data...
Loaded DGL Graph
adj list created
CPU times: user 2min 44s, sys: 11.3 s, total: 2min 55s
Wall time: 2min 54s


In [17]:
# %%time
# dgl_scratch = dgl.from_networkx(scratch_G)

# Train Model

In [18]:
%%time
# Training the Model. GPU ~ 00:00:40
with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)
model, pred = train(dgl_G, feat_data, adj_list, **model_cfg)

# Put everything on CPU
model = model.to('cpu')
pred = pred.to('cpu')


Train pos edge: 26413580
Validation pos edge: 518464
Cuda enabled: True

Training starts:
In epoch 1 batch 1, loss: 0.7928540110588074
In epoch 1 batch 6, loss: 0.6777392625808716
In epoch 1 batch 11, loss: 0.49362197518348694
In epoch 1 batch 16, loss: 0.22828425467014313
In epoch 1 batch 21, loss: 0.10402873158454895

Epoch 1 AUC:  0.4493406049153463
In epoch 2 batch 1, loss: 0.1185450330376625
In epoch 2 batch 6, loss: 0.14984646439552307
In epoch 2 batch 11, loss: 0.13696223497390747
In epoch 2 batch 16, loss: 0.12090416997671127
In epoch 2 batch 21, loss: 0.10547196120023727

Epoch 2 AUC:  0.4792098082859871
In epoch 3 batch 1, loss: 0.10279714316129684
In epoch 3 batch 6, loss: 0.1088569387793541
In epoch 3 batch 11, loss: 0.10724322497844696
In epoch 3 batch 16, loss: 0.10788120329380035
In epoch 3 batch 21, loss: 0.10887868702411652

Epoch 3 AUC:  0.5075558653971188
In epoch 4 batch 1, loss: 0.11191188544034958
In epoch 4 batch 6, loss: 0.10431034862995148
In epoch 4 batch 11, 

In [19]:
# Create Embeddings
z = model(dgl_G, feat_data)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [163]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

# Get Random Playlist

In [22]:
'''
Get a list of eligible slice files (first 10000 playlists)
thelist: list of directories
'''
def get_eligible(thelist):
    eligible = []
    for x in thelist:
        nums = pd.Series(x.strip('mpd.slice.json').split('-')).astype(int)
        if nums[0] <= 9999:
            eligible.append(x)
    return eligible

In [141]:
'''
Gets random playlist
Gets the track names of the original tracks in the playlist
'''
def get_playlist():
    data_path = (os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/'))
    file_samp = np.random.choice(get_eligible(pd.Series(os.listdir(os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/')))), replace=True)
    fname = os.path.join(data_path, file_samp)
    with open(fname) as f:
        data = json.load(f)
        item = np.random.choice(data['playlists'])
    print(fname)    
    print('Playlist ID:', item['pid'])
    print('Playlist Length:', len(item['tracks']))
    
    # Get track names---artist
    original_tracks = []
    for i in range(len(item['tracks'])):
        name = item['tracks'][i]['track_name']+'---'+item['tracks'][i]['artist_name']
        original_tracks.append(name)
        
    # Get track uris
    seeds = []
    for i in item['tracks']:
        uri = i['track_uri'].split(':')[-1]
        seeds.append(uri)
        
    return item, original_tracks, seeds

In [142]:
item, original_tracks, seeds = get_playlist()
original_tracks

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.9000-9999.json
Playlist ID: 9266
Playlist Length: 11


['Celebraré Tu Amor---Jesús Adrián Romero',
 'Alzad Oh Puertas---Jesús Adrián Romero',
 'Tu Nos Creaste---Jesús Adrián Romero',
 'Abre Los Cielos---Jesús Adrián Romero',
 'Un Destello De Tu Gloria---Jesús Adrián Romero',
 'Al Estar Ante Ti---Jesús Adrián Romero & Alejandro Del Bosque',
 'No Me Soltarás---Rojo',
 'Vino Celestial---Miel San Marcos',
 'El Santo De Israel---Miel San Marcos',
 'Vino Celestial---Miel San Marcos',
 'Enamorado de Ti---René González']

# Recommend

In [183]:
'''
Creates dictionary of highest scored recommendation (of songs not in playlist) for each song in playlist
seeds: list of track uris from user's playlist
dgl_G: DGL Graph
z: embeddings generated from model
pred: predictor from model
feat_data: matrix of feature data
'''
def recommend(seeds, dgl_G, z, pred, neigh, feat_data):

    listed = list(uri_map) #parse through uri map for uri --> integer

    score_dict = defaultdict(dict)
    for s in seeds:
        s = uri_map[s]
        _, candidates = dgl_G.out_edges(s, form='uv')
        s_embed = z[s].unsqueeze(dim=0)
        edge_embeds = [torch.cat([s_embed, z[c.item()].unsqueeze(dim=0)],1) for c in candidates]
        print('Node Value:', s, 'Possible Recs:', len(edge_embeds))
        edge_embeds = torch.cat(edge_embeds, 0)
        scores = pred.W2(F.relu(pred.W1(edge_embeds))).squeeze(1)
        val = list(zip(candidates.detach().numpy(), scores.detach().numpy()))
        val.sort(key=lambda x:x[1], reverse=True)
        
        # Make sure the song is not already in the playlist
        # score_dict[s] = val[0]
        inc = 0
        while True and inc < len(val):
            if listed[val[inc][0]] not in seeds:
                score_dict[s] = val[inc][0]
                break
            if inc == (len(val) - 1):
                # If no co-occurence, use 5-NN based on features -- COLD START
                print('None Found, Using Feature Data Instead')
                closest = neigh.kneighbors(feat_data[[s]], 25, return_distance=False)[0]
                for i in closest:
                    if listed[i] not in seeds:
                        score_dict[s] = i
                        break
                break
                    
            else:
                inc += 1
                
    # Get uris            
    uri_recs = []
    for i in score_dict.keys():
        cur_uri = listed[score_dict[i]]
        uri_recs.append(cur_uri)
        
    return uri_recs

In [184]:
uri_recs = recommend(seeds, dgl_G, z, pred, neigh, feat_data)
uri_recs

Node Value: 103404 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 102021 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 103838 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 131896 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 10733 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 32115 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 46453 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 50303 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 63782 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 50303 Possible Recs: 9
None Found, Using Feature Data Instead
Node Value: 64778 Possible Recs: 9
None Found, Using Feature Data Instead


['3mDL7gITYAea1T2nJBs5Tk',
 '7IN4zUEZQJkyATFepIrNWP',
 '6IL6dgzFkfiRgHORMlxkMf',
 '2eHFk44RlGGfcJhYioirFR',
 '641BnSVI1ydg9RWBSKvaXv',
 '4PrzTV80lUwZlZbvd2oswH',
 '2BJMHc2oMWUv0bimChLsvG',
 '0N6W6IJi7zYAUhKJULGDaH',
 '6CE9kx5HJSH2D836t5QouD',
 '1yw66ZPsfqblI4PFcEdIKe']

In [41]:
listed = list(uri_map) #parse through uri map for uri --> integer
listed[155421]

'77aasEzxEXE13suFG6npRE'

In [42]:
adj_list[155421]

{5832,
 13024,
 20176,
 24311,
 31689,
 34816,
 42293,
 47402,
 65694,
 101105,
 102554,
 114408,
 120586,
 137468,
 138034,
 150486,
 151243,
 159998,
 162902}

In [9]:
dgl_G.out_edges(155421, form='uv')

(tensor([155421, 155421, 155421, 155421, 155421, 155421, 155421, 155421, 155421,
         155421, 155421, 155421, 155421, 155421, 155421, 155421, 155421, 155421,
         155421]),
 tensor([159998, 137468, 101105, 102554,  47402,   5832, 162902,  24311, 138034,
         150486,  65694, 151243,  34816,  20176,  42293, 120586,  13024,  31689,
         114408]))

# Translate Codes with Spotify API

In [185]:
client_id = 'ad2536ed7a914d66b89b80fb3a500787'
client_secret = '8c5f45fb008d4bc5bf909ec46d076b65'

spotify = SpotifyAPI(client_id, client_secret)

In [186]:
def get_data_spotify(query, api, num):
    chunk = api.get_resource(query, 'tracks', 'v1')
    return chunk

In [187]:
def get_rec_names(uri_recs, api, sleep_time):
    rec_track_names = []
    for i in uri_recs:
        one = get_data_spotify(i, api, 1)
        trackname = one['tracks'][0]['name']

        firstartist = one['tracks'][0]['artists'][0]['name']

        the_rec = trackname+'---'+firstartist
        rec_track_names.append(the_rec)


        time.sleep(sleep_time)
    return rec_track_names

In [188]:
%%time
rec_track_names = get_rec_names(uri_recs, spotify, 2)

CPU times: user 348 ms, sys: 44 ms, total: 392 ms
Wall time: 22 s


In [189]:
original_tracks

['Celebraré Tu Amor---Jesús Adrián Romero',
 'Alzad Oh Puertas---Jesús Adrián Romero',
 'Tu Nos Creaste---Jesús Adrián Romero',
 'Abre Los Cielos---Jesús Adrián Romero',
 'Un Destello De Tu Gloria---Jesús Adrián Romero',
 'Al Estar Ante Ti---Jesús Adrián Romero & Alejandro Del Bosque',
 'No Me Soltarás---Rojo',
 'Vino Celestial---Miel San Marcos',
 'El Santo De Israel---Miel San Marcos',
 'Vino Celestial---Miel San Marcos',
 'Enamorado de Ti---René González']

In [190]:
pd.Series(rec_track_names).value_counts()

Voodoo - Live---Godsmack                                                     1
Feel Good Inc. - Live at Manchester Opera House---Gorillaz                   1
Can't Get Enough - Live At The Wembley Arena, London / 2010---Bad Company    1
No Basta---Franco De Vita                                                    1
Ten Thousand Words---The Avett Brothers                                      1
---                                                                          1
Not Ashamed---Jeremy Camp                                                    1
Hokages Furneral - Dance Mix---The Game Music Committee                      1
All For Love---Planetshakers                                                 1
Make Me Believe in You---Curtis Mayfield                                     1
dtype: int64